In [ ]:
# read the data
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import RandomForestRegressor
import matplotlib
from matplotlib import pylab as plt
import pandas as pd
import numpy as np

# Let's load the data
df = pd.read_csv('online_shoppers_intention.csv')

# drop the target variable
y = df['Revenue']
df.drop(columns=['Revenue'],inplace=True)

# features that does not need to be preprocess: 'BounceRates', 'ExitRates'

# OneHot Encoder.
df_cat = df[['OperatingSystems','Browser','Region','TrafficType','VisitorType','Weekend']]

df_imp = pd.DataFrame(imp.fit_transform(df_cat),columns = ['OperatingSystems','Browser','Region','TrafficType',
                                                           'VisitorType','Weekend'] )

hot_enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
all_hot = hot_enc.fit_transform(df_imp[['OperatingSystems','Browser','Region','TrafficType','VisitorType','Weekend']])
df_alloh = pd.DataFrame(all_hot, columns = hot_enc.get_feature_names(['OperatingSystems','Browser','Region','TrafficType',
                                                                      'VisitorType','Weekend']))

# Ordinal Encoder
df_ord = df[['Month']]
ord_enc = OrdinalEncoder(categories = [['Feb', 'Mar', 'May', 'June','Jul','Aug','Sep','Oct','Nov','Dec']]) 
df_ord = pd.DataFrame(ord_enc.fit_transform(df_ord[['Month']]),columns = ['Month'])



In [ ]:
# Standard Scalar
df_con = df[['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 'ProductRelated', 
             'ProductRelated_Duration','PageValues', 'SpecialDay']]
sta_scaler = StandardScaler()

df_num = pd.DataFrame(sta_scaler.fit_transform(df_con[['Administrative', 'Administrative_Duration', 'Informational',
                                                       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
                                                       'PageValues', 'SpecialDay']]), 
                      columns = ['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                                 'ProductRelated', 'ProductRelated_Duration','PageValues', 'SpecialDay'])


In [ ]:
# Preprocess the target variable ('Revenue')                                                              
le = LabelEncoder()
target_le = le.fit_transform(y.values.ravel())
df_rev = pd.DataFrame(target_le,columns = ['Revenue'])
df_rev.head()

# Concatenate all dataframe into one large dataframe
df_data = pd.concat([df_alloh,df_ord,df_num,df_rev],axis=1)
df_data